In [30]:
# --- Instalación de librerías necesarias ---
!pip install feedparser requests

# --- Importaciones ---
import feedparser
import pandas as pd
import requests

# === FUNCIONES ===

# 1. Extraer noticias desde un RSS
def get_news_from_rss(feed_url, source_name):
    feed = feedparser.parse(feed_url)
    news_items = []
    for entry in feed.entries:
        news_items.append({
            "title": entry.get("title", ""),
            "published": entry.get("published", ""),
            "link": entry.get("link", ""),
            "summary": entry.get("summary", ""),
            "source": source_name
        })
    return pd.DataFrame(news_items)

# 2. Filtrar noticias por relevancia (temas clave)
def filter_relevant_news(news_df):
    # Definir las palabras clave relevantes
    keywords = ['economía', 'política', 'finanzas', 'gobierno', 'presupuesto', 'inversión', 'mercado']

    # Filtrar noticias que contienen las palabras clave en el título o resumen
    filtered_news = news_df[news_df['title'].str.contains('|'.join(keywords), case=False, na=False) |
                            news_df['summary'].str.contains('|'.join(keywords), case=False, na=False)]
    return filtered_news.head(50)  # Obtener las primeras 50 noticias relevantes

# 3. Obtener clima desde Open-Meteo
def get_weather_data(lat, lon, city):
    url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current_weather=true"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json().get("current_weather", {})
        return pd.DataFrame([{
            "city": city,
            "temperature": data.get("temperature"),
            "windspeed": data.get("windspeed"),
            "weathercode": data.get("weathercode"),
            "time": data.get("time")
        }])
    else:
        print(f"❌ Error al obtener clima de {city}")
        return pd.DataFrame()

# 4. Obtener tipo de cambio desde e-api.net.pe
def get_exchange_rate():
    url = "https://free.e-api.net.pe/tipo-cambio/today.json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return pd.DataFrame([{
            "fecha": data.get("fecha"),
            "compra": data.get("compra"),
            "venta": data.get("venta"),
            "origen": data.get("origen")
        }])
    else:
        print("❌ Error al obtener tipo de cambio")
        return pd.DataFrame()

# 5. Verificar completitud de los datos
def check_completeness(df, name):
    print(f"\n🔎 Verificando completitud de {name}:")
    print(df.isnull().sum())

# 6. Verificar frescura de los datos
def check_data_freshness(df, last_updated_column, current_date):
    print(f"\n🔎 Verificando frescura de los datos (fecha actual: {current_date}):")
    df[last_updated_column] = pd.to_datetime(df[last_updated_column], errors='coerce')
    last_updated = df[last_updated_column].max()

    # Convertir ambos a tz-naive antes de restar
    last_updated = last_updated.tz_localize(None)  # Eliminar zona horaria
    current_date = current_date.tz_localize(None)  # Eliminar zona horaria

    freshness = (current_date - last_updated).days
    print(f"Última actualización: {last_updated} - Días desde última actualización: {freshness}")
    return freshness

# 7. Verificar relevancia para el negocio
def check_business_relevance(df, condition_column, condition_value):
    print("\n🔎 Verificando relevancia para el negocio:")
    relevant_data = df[df[condition_column] > condition_value]
    print(f"Datos relevantes para el negocio (condición {condition_column} > {condition_value}):")
    print(relevant_data)

# === FUENTES RSS ===
feeds = {
    "El Comercio": "https://elcomercio.pe/arcio/rss/",
    "La República": "https://larepublica.pe/rss/",
    "Gestión": "https://gestion.pe/arcio/rss/"
}

# === EXTRACCIÓN DE DATOS ===

# Noticias desde RSS
news_frames = [get_news_from_rss(url, name) for name, url in feeds.items()]
news_df = pd.concat(news_frames, ignore_index=True)
print("✅ Noticias de medios peruanos:")
# Filtrar noticias relevantes por tema y mostrar solo las 50 más relevantes
relevant_news_df = filter_relevant_news(news_df)
print(relevant_news_df[['title', 'source', 'published']].head(50))

# Clima: Lima, Cusco y Arequipa
weather_df = pd.concat([
    get_weather_data(-12.0464, -77.0428, "Lima"),
    get_weather_data(-13.5319, -71.9675, "Cusco"),
    get_weather_data(-16.4090, -71.5375, "Arequipa")
], ignore_index=True)
print("\n✅ Clima de ciudades:")
print(weather_df.head())

# Tipo de cambio
exchange_df = get_exchange_rate()
print("\n✅ Tipo de cambio PEN:")
print(exchange_df.head())

# === VERIFICACIÓN DE CALIDAD DE DATOS ===
print("\n===== VERIFICACIÓN DE CALIDAD DE DATOS =====")

# Completitud de los datos
check_completeness(news_df, "Noticias")
check_completeness(weather_df, "Clima")
check_completeness(exchange_df, "Tipo de Cambio")

# === VERIFICACIÓN DE FRESCURA Y RELEVANCIA ===
current_date = pd.to_datetime('2025-04-04')

# Ejemplo de llamada a la función para verificar frescura de los datos
check_data_freshness(news_df, 'published', current_date)

# Verificar relevancia para el negocio (ejemplo: clima > 20°C para productos fríos)
check_business_relevance(weather_df, 'temperature', 20)

# Justificación de la elección de los indicadores de calidad:

print("\n===== JUSTIFICACIÓN DE LA ELECCIÓN DE LOS INDICADORES =====")

# 1. **Completitud**:
# La completitud es fundamental para asegurarse de que los datos no tengan valores faltantes en columnas clave (como el título de noticias, fecha, etc.). Si la completitud es baja, los resultados no serán confiables para análisis.
print("Completitud: Asegura que no haya valores nulos en datos cruciales como los títulos de noticias, las fechas de las publicaciones y el contenido del clima. La falta de estos datos clave impactaría negativamente en los modelos analíticos.")

# 2. **Frescura**:
# La frescura mide cuán recientes son los datos. Si los datos no se actualizan con la frecuencia necesaria, podrían no reflejar las condiciones actuales del mercado, lo que afectaría la precisión de las predicciones de demanda y estrategias de precios.
print("Frescura: En un proyecto de forecast de demanda, necesitamos datos frescos para reflejar las condiciones actuales. Si las noticias sobre cambios en la economía o clima son antiguas, podrían no ser relevantes para los análisis.")

# 3. **Relevancia para el negocio**:
# La relevancia se refiere a cuán bien los datos se alinean con los objetivos de negocio, como la demanda de productos fríos en función de la temperatura. Este indicador es clave para garantizar que solo se utilicen datos pertinentes.
print("Relevancia para el negocio: Solo los datos relevantes para la empresa deben ser analizados. Por ejemplo, en el caso de productos fríos, datos de clima con temperaturas superiores a 20°C serían más relevantes.")

# 4. **Confiabilidad**:
# La confiabilidad mide la consistencia y exactitud de los datos. Si la fuente no es confiable, el análisis que se realice será erróneo.
print("Confiabilidad: Usar fuentes confiables (como noticias verificadas y fuentes oficiales de tipo de cambio) es crucial para evitar el uso de datos sesgados o incorrectos.")

# --- Fin del código ---


✅ Noticias de medios peruanos:
                                                 title       source  \
4    Precio del dólar en Perú: consulta el tipo de ...  El Comercio   
12                                 ¡A ver, un aplauso!  El Comercio   
16                                   Altitud y talento  El Comercio   
54   Congreso: presentan 4 mociones para interpelar...  El Comercio   
61   Gobierno decreta 3 días no laborables compensa...  El Comercio   
84   Wall Street cierra con fuerte baja tras arance...  El Comercio   
91   Milei anunciaría esta noche una negociación co...  El Comercio   
101  Gobierno destina S/ 526.6 millones para aument...      Gestión   
108  ¿Será contratado de manera definitiva? Hasta c...      Gestión   
113  Trump ultima sus nuevos aranceles y el mundo s...      Gestión   
116  Teleféricos en Lima: ATU replantea proyecto, p...      Gestión   
117  La nueva inversión de César Acuña: Universidad...      Gestión   
126  Gobierno buscará dialogar con medios de c